# Alan Data

i.e. the big one.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from edges_cal import modelling as mdl
from pathlib import Path

import mcmc_utils as mutl
from edges_estimate.eor_models import AbsorptionProfile
from edges_estimate.fitting import SemiLinearFit
from getdist import plots

import alan_data_utils as utils
import alan_data as adata
import glob
import run_alan_data_mcmc as mcmc
from scipy import optimize
import notebook_utils as nutl
import pickle
from multiprocessing import Pool
from astropy import units as u
from edges_cal.modelling import LinLog
plt.ion()

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
lk0 = mcmc.get_likelihood(
    nterms_fg=5, 
    fix_tau=False, 
    simultaneous=True, 
    cterms=6, 
    wterms=5, 
    smooth=32, 
    tns_width=500, 
    fit_cterms=6,
    fit_wterms=5, 
    nscale=0, 
    ndelay=0, 
    est_tns=np.zeros(6), 
    ignore_sources=(), 
    as_sim=('short', 'open'), 
    s11_sys=(), 
    antsim=False, 
    sim_sky=False, 
    add_noise=True
)

S11 SYSTEMATICS:  {}
6 6 6
('short', 'open')


In [3]:
lk1 = mcmc.get_likelihood(
    nterms_fg=5, 
    fix_tau=False, 
    simultaneous=True, 
    cterms=6, 
    wterms=5, 
    smooth=32, 
    tns_width=500, 
    fit_cterms=6,
    fit_wterms=5, 
    nscale=1, 
    ndelay=1, 
    est_tns=np.zeros(6), 
    ignore_sources=(), 
    as_sim=('short', 'open'), 
    s11_sys=(), 
    antsim=False, 
    sim_sky=False, 
    add_noise=True
)

S11 SYSTEMATICS:  {}
6 6 6
('short', 'open')


In [4]:
lk0.partial_linear_model()

(2721.463904671679,
 [(183.80910854113037,
   -2.935722563058962,
   1.4028655316834,
   -0.4575999425184004,
   0.11846589621226686),
  (-9.406529304719141,
   -3.7014626835034283,
   -7.438628190559872,
   4.522425964867686,
   -2.58074125066428),
  (10.650995672025465,
   8.740772769222461,
   -3.6588533640418874,
   -1.5498672569311696,
   0.7847466874554903),
  (301.5317196211456,
   -0.005616770336702949,
   0.17171485790407462,
   -0.11449721834690772,
   -0.19037371176026185,
   0.11263375790963598),
  (1748.0300725535535,
   -121.00733305038814,
   -66.7031572695831,
   49.98127417529596,
   158.47930685923586),
  array([[ 1.83805090e+02, -2.93241230e+00,  1.42431027e+00,
          -4.71096910e-01,  9.47763028e-02, -9.40949228e+00,
          -3.70144731e+00, -7.42125864e+00,  4.52199088e+00,
          -2.59483994e+00,  1.06546197e+01,  8.74039849e+00,
          -3.66795164e+00, -1.54916906e+00,  7.92585149e-01,
           3.01528692e+02,  5.22240938e-04,  1.89596931e-01,
     

In [5]:
lk1.partial_linear_model()

(2721.463904671679,
 [(183.80910854113037,
   -2.935722563058962,
   1.4028655316834,
   -0.4575999425184004,
   0.11846589621226686),
  (-9.406529304719141,
   -3.7014626835034283,
   -7.438628190559872,
   4.522425964867686,
   -2.58074125066428),
  (10.650995672025465,
   8.740772769222461,
   -3.6588533640418874,
   -1.5498672569311696,
   0.7847466874554903),
  (301.5317196211456,
   -0.005616770336702949,
   0.17171485790407462,
   -0.11449721834690772,
   -0.19037371176026185,
   0.11263375790963598),
  (1748.0300725535535,
   -121.00733305038814,
   -66.7031572695831,
   49.98127417529596,
   158.47930685923586),
  array([[ 1.83809894e+02, -2.92241013e+00,  1.36969529e+00,
          -4.81762491e-01,  1.70216093e-01, -9.40385304e+00,
          -3.69745771e+00, -7.45081713e+00,  4.50981203e+00,
          -2.56794130e+00,  1.06540258e+01,  8.73679288e+00,
          -3.67764547e+00, -1.54316090e+00,  8.05840466e-01,
           3.01531142e+02,  5.64266165e-03,  1.46156372e-01,
     

In [6]:
lk0.data.keys()

dict_keys(['q', 'T', 'k0', 'data_variance', 'loss', 'bm_corr', 'loss_temp', 'gamma_src', 'gamma_rcv', 'raw_basis'])

In [18]:
for k in lk0.data:
    v0 = lk0.data[k]
    v1 = lk1.data[k]
    
    if isinstance(v0, dict):
        for kk in v0:
            vv0 = v0[kk]
            vv1 = v1[kk]
            
            if not np.all(vv0==vv1):
                print(k, kk)
    else:
        if not np.all(v0==v1):
            print(k, kk)

q open
q short


In [16]:
lk0.data['data_variance']['ant'] - lk1.data['data_variance']['ant']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])